### Processamento de Línguagem Natural  
  
### Alexandre Ribeiro de Lima  nº 74.603  

28/03/2021 



---

# **Tarefa:** Detecção de spam com corpus de dados reais

Nessa tarefa, vamos trabalhar com parte do corpus de mensagens de e-mail da Enron.

As mensagens estão em arquivos de texto curtos em inglês (principalmente, mas não só) e têm anotações manuais no título e na primeira linha de texto com sua etiqueta como "spam" (1) ou "ham" (0). 

Aqui está uma mensagem de exemplo:


---


0

Subject: meter 1517 - jan 1999
george ,
i need the following done :
jan 13
zero out 012 - 27049 - 02 - 001 receipt package id 2666
allocate flow of 149 to 012 - 64610 - 02 - 055 deliv package id 392
jan 26
zero out 012 - 27049 - 02 - 001 receipt package id 3011
zero out 012 - 64610 - 02 - 055 deliv package id 392
these were buybacks that were incorrectly nominated to transport contracts
( ect 201 receipt )
let me know when this is done
hc

---

Observe o 0 na primeira linha. Ele indica que a mensagem foi etiquetada como *ham*.



**1. Abrir Arquivo**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Enron.zip to Enron.zip


In [ ]:
# Descompactar arquivo
import os
import zipfile

local_zip = "/content/Enron.zip"

zip_ref = zipfile.ZipFile(local_zip, "r")

zip_ref.extractall("/content/Enron")
zip_ref.close()

In [ ]:
!ls

Enron  Enron.zip  sample_data


In [ ]:
# Criar paths para cada arquivo de texto

import os
from glob import glob

dir_text = "/content/Enron/"

glob_text = os.path.join(dir_text, "*.txt")

path_text = sorted(glob(glob_text))

print(len(path_text))

print("\n", path_text[1])

5157

 /content/Enron/0002.1999-12-13.farmer.ham.txt


In [ ]:
# Criar listas de cada mensagem de email do arquivo Enrom
lst_lista=[]

for i in path_text:
  arquivo = open(i, "r")
  arq = arquivo.read()
  lst_lista.append([arq])
  arquivo.close()
print(lst_lista[0])


['0\nSubject: christmas tree farm pictures\n']


In [ ]:
# Separar o rótulo das mensagem e armazená-las em listas diferentes
lst_classes = []
lst_texto = []
for lst in lst_lista:
  lst_classes.append(lst[0][0])
  lst_texto.append(lst[0][1:])
print(lst_classes[:6])
print(lst_texto[0])
   

['0', '0', '0', '0', '0', '1']

Subject: christmas tree farm pictures



In [ ]:
# Criar o corpus rotulado
corpus = list(zip(lst_texto,lst_classes))
print(corpus[0])

('\nSubject: christmas tree farm pictures\n', '0')


**2. Pré-Processamento**

In [ ]:
# Importar bibliotecas
import itertools
from collections import Counter
from numpy import prod
from math import log

import nltk
nltk.download('stopwords')
nltk.download('punkt')
stops = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Criar as funções de pré-processamento

def tokenizar(str_texto):
    return word_tokenize(str_texto, language='english')


def sem_stops(lst_palavras):
    return [p for p in lst_palavras if p not in stops]


def limpar(lista):
    return [i.lower() for i in lista if i.isalpha()]


def pre_processar(str_texto):
    return stemizar(sem_stops(limpar(tokenizar(str_texto))))


def achatar(lista):
    return list(itertools.chain(*lista))

def stemizar(lista):
    return [stemmer.stem(i) for i in lista]



In [ ]:
# Pré-Processar o corpus

corpus = [(pre_processar(i[0]), i[1]) for i in corpus]
print(corpus[0])

(['subject', 'christma', 'tree', 'farm', 'pictur'], '0')


**3. Dividir o corpus pré-processado em treinamento (80%) e teste (20%)**

In [ ]:
# Separação dos documentos
all_ham = [i[0] for i in corpus if i[1]=="0"]
all_spam =[i[0] for i in corpus if i[1]=="1"]

# Criar o vacabulário com palavras únicas
vocab = set (achatar(all_ham)) | set(achatar(all_spam))

In [ ]:
# Dividir documentos em treino e teste
# 80% para treinamento e 20% para teste
from sklearn.model_selection import train_test_split
import numpy as np

train_ham, test_ham = train_test_split(all_ham, test_size=0.2, shuffle=False)

train_spam,test_spam = train_test_split(all_spam,test_size=0.2, shuffle=False)


**4. Realizar as contagens no corpus de treinamento**

In [ ]:
# Contagem do vocabulário
n_vocab = len(vocab)
print(n_vocab)

37892


In [ ]:
# Contagem das classes spam e ham
n_train_ham = len(train_ham)
n_train_spam = len(train_spam)
# Contagem do total das classes
n_total_train = n_train_ham + n_train_spam


print("Numero de exemplos de treinamemto da classe ham =", len(train_ham))
print("\nNumero de exemplos de teste da classe ham =", len(test_ham))
print("\nNumero de exemplos de treinamemto da classe spam =", len(train_spam))
print("\nNumero de exemplos de teste da classe spam =", len(test_spam))
print("\nNúmero de exemplos totais de treinamento = ", n_total_train)

Numero de exemplos de treinamemto da classe ham = 2937

Numero de exemplos de teste da classe ham = 735

Numero de exemplos de treinamemto da classe spam = 1188

Numero de exemplos de teste da classe spam = 297

Número de exemplos totais de treinamento =  4125


In [ ]:
# Contagem dos atributos (cada token corresponde a um atributo)
# Contagens de cada palavra (dicionários de ocorrências)
tokens_ham = achatar(all_ham)
cont_ham = Counter(tokens_ham)
tokens_spam = achatar(all_spam)
cont_spam = Counter(tokens_spam)

# Número total de palavras em cada classe
n_tokens_ham = sum(cont_ham.values())
n_tokens_spam = sum(cont_spam.values())

print("Contagem de cada palavra na classe ham: ", len(cont_ham))
print("\nContagem de cada palavra na classe spam: ", len(cont_spam))
print("\nContagem total de palavras na clase ham: ", n_tokens_ham)
print("\nContagem total dos palavrasda classe spam: ", n_tokens_spam)


Contagem de cada palavra na classe ham:  12360

Contagem de cada palavra na classe spam:  31492

Contagem total de palavras na clase ham:  323901

Contagem total dos palavrasda classe spam:  177054


**5. Calcular as probabilidades relacionadas às contagens**

O cálculo das probabilidades estão dentro da função Bayes().

**6. Implementar uma função Bayes() que receba uma mensagem e devolva a probabilidade dela ser classificada como spam ou ham**

In [ ]:
def Bayes (mensagem):

  # testa e exclui palavras que não aparecem no vocabulário de treinamento
  tokens_teste = [i for i in mensagem if i in vocab]

  # Calcula a probabilidade da classe ham
  prob_classe_ham = n_train_ham / n_total_train
  #Suavização de Laplece para classe ham
  prob_f_dada_ham = [(cont_ham[i]+1)/(n_tokens_ham + n_vocab)for i in tokens_teste]
  # cálculo da probabilidade do email ser ham
  prob_ham = log(prob_classe_ham) + sum(log(i) for i in prob_f_dada_ham)

  # Calcula a probabilidade da classe spam
  prob_classe_spam = n_train_spam / n_total_train
  #Suavização de Laplece para classe spam
  prob_f_dada_spam = [(cont_spam[i]+1)/(n_tokens_spam + n_vocab)for i in tokens_teste]
  # cálculo da probabilidade do email ser ham
  prob_spam = log(prob_classe_spam) + sum(log(i) for i in prob_f_dada_spam)

  return prob_spam, prob_ham


In [ ]:
#testando a função Bayes()
test_1=Bayes(train_ham[0])
print(test_1)

(-43.7462888162411, -44.55875654679888)


**7. Classificar todo o corpus de teste passando cada mensagem pela função Bayes()**

In [ ]:
# Criar função que classifica um conjunto de dados com a função Bayes() 
# e retorna um vetor com os emails classificados em 1(spam) ou 0(ham)

def classificadorBayes(conjunto_dados):

  vetor= []
  result = []

  for i in conjunto_dados:
    vetor.append(Bayes(i))

  
  for i in vetor:
    if i[0] > i[1]:
      result.append(1)
    else:
      result.append(0)

  return result

In [ ]:
# instancia o classificador com os dados de teste de spam
# e retorna um vetor de classificação para os dados de spam
classificar_spam =classificadorBayes(test_spam)

# instancia o classificador com os dados de teste de ham
# e retorna um vetor de classificação para os dados de ham
classificar_ham = classificadorBayes(test_ham)

**8. Criar a matriz de confusão do classificador**

In [ ]:
#Criar os conjuntos de dados separados
import numpy as np

Spam_original = achatar([np.ones(len(test_spam), dtype=int)])
Spam_previsto= classificar_spam
Ham_original = achatar([np.zeros(len(test_ham), dtype=int)])
Ham_previsto = classificar_ham

In [ ]:
# Criar o conjunto de dados originais e os previstos
dados_originais = Spam_original + Ham_original

dados_previstos = Spam_previsto + Ham_previsto

In [ ]:
#Verificar o cumprimento dos vetores
print(len(dados_originais))
print(len(dados_previstos))

1032
1032


In [ ]:
#Criar a Matriz de confusão
from sklearn.metrics import confusion_matrix

VN, FP, FN, VP = confusion_matrix(dados_originais, dados_previstos).ravel()
print(VN, FP, FN, VP)

727 8 8 289


**9. Calcular a performance do classificador**

In [ ]:
# Calcular a Precisão
precision = VP / (VP + FP)
print("A precisão do modelo é de: ", precision)

# Calcular a Cobertura ou Recall
cobertura = VP/(VP + FN)
print("\nA cobertura ou recall do modelo é de:", cobertura)
# Calcular a Acurácia
acuracia = (VP + VN)/(VP + FP + VN + FN)
print("\nA acurácia do modelo é de: ", acuracia)
# Calcular a Medida_F ou F-score
medida_F = 2*((precision*cobertura)/(precision+cobertura))
print("\nA medida_F ou F-score do modelo é de: ", medida_F)

A precisão do modelo é de:  0.9730639730639731

A cobertura ou recall do modelo é de: 0.9730639730639731

A acurácia do modelo é de:  0.9844961240310077

A medida_F ou F-score do modelo é de:  0.9730639730639731
